# Scraping the Dextoools trade data

In [1]:
import requests
from bs4 import BeautifulSoup

# specify the URL of the WETH/USDT page
url = 'https://www.dextools.io/app/en/ether/pair-explorer/0x4e68ccd3e89f51c3074ca5072bbac773960dfa36'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

# send a GET request to the URL
response = requests.get(url, headers=headers)

# parse the HTML content of the page using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# find the main content of the page by looking for the first <div> tag with class 'mw-parser-output'
div_table_frame = soup.find("div", {"class":"ng-tns-c167-3"})
# content_div = div_content_text.find('div', {'class': 'mw-parser-output'})

# # get all the text content from the page by concatenating the text of all <p> tags within the content div
# text_content = ''
# for p in content_div.find_all('p'):
#     text_content += p.get_text()

# # print the text content
# print(text_content)

In [2]:
soup.find_all("p")##hmm this again produce error, continue here

[<p>You must have JavaScript enabled to run this app!</p>]

In [1]:
import requests
from bs4 import BeautifulSoup
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# specify the URL of the WETH/USDT page
url = 'https://www.dextools.io/app/en/ether/pair-explorer/0x4e68ccd3e89f51c3074ca5072bbac773960dfa36'

options = Options()
#options.add_experimental_option("detach", True)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                          options=options)

driver.get(url)
#driver.maximize_window()

#wait = WebDriverWait(driver, 15).until(lambda driver: driver.find_element("xpath", "//div[@role='table']"))#.is_displayed())

# table = driver.find_elements("xpath", "//div[@role='table']")
# for i in table:
#     print(i.get_attribute("innerHTML"))
# table = driver.find_element("xpath", "//datatable-body[1]")
# print(table.get_attribute("innerHTML"))


# I suppose this is one way, get the content into soup and work on it from there
# content = driver.page_source
# soup = BeautifulSoup(content, 'html.parser')
# print(soup)

# The Stack overflow way
# get table
tableElement = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.TAG_NAME, 'ngx-datatable'))
)

# scroll into table view
driver.execute_script("arguments[0].scrollIntoView();", tableElement)

#probably here I should check no bots!

# scrolling through the table body to the bottom
tableBodyelement = tableElement.find_element(By.TAG_NAME, "datatable-body-cell")#   .find_element_by_tag_name('datatable-body-cell')
driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight)", tableBodyelement)


rowWrapper = tableElement.find_elements(By.TAG_NAME, 'datatable-row-wrapper')

#for i in rowWrapper:
#    print(i.get_attribute("innerHTML"))


for row in rowWrapper:
    cells = row.find_elements(By.TAG_NAME, 'datatable-body-cell')
    date = cells[0].text
    type = cells[1].text
    price_usd = cells[2].text
    price_eth = cells[3].text
    amount_coin_1 = cells[4].text
    total_coin_2 = cells[5].text  #the naming here should be done better, it will depend on the pairs I will be trading
    total_chain_coin = cells[6].text
    maker = cells[7].find_element(By.TAG_NAME, 'a').get_attribute('href')
    print(date, type, price_usd, price_eth, amount_coin_1, total_coin_2, total_chain_coin, maker)
    print('----')    

#print(tableElement.get_attribute("innerHTML"))
#driver.quit()


#next continue here. Tydy up the code, insert the whole tyhing into while loop, save into dataframe.
#Also TODO: filter bot activity



2023-03-08 16:37:23 buy $1,560.30 1 4.47293 7,000.00 4.47293 https://etherscan.io/address/0x277834ffb8f94908ee6ea5eeae28bdbf995ef4e0
----
2023-03-08 16:37:23 buy $1,560.14 1 0.6773 1,060.00 0.6773 https://etherscan.io/address/0x935edfa4058fb0df08ba6ce050b52190ad08e4ef
----
2023-03-08 16:36:23 buy $1,560.14 1 36.3569 56,871.5 36.3569 https://etherscan.io/address/0x55e2cbd1e9c5c7e8a18eacaafc7b2e050d4b2827
----
2023-03-08 16:36:11 buy $1,558.89 1 3.16579 4,950.00 3.16579 https://etherscan.io/address/0x043ae83ed1a789183405b5b59fe39a127eaaf28a
----
2023-03-08 16:34:11 buy $1,558.74 1 24.1023 37,675.1 24.1023 https://etherscan.io/address/0x06209508093e5255bfedf646f517af9d3f7e7158
----
2023-03-08 16:31:23 buy $1,557.96 1 48.8343 76,274.4 48.8343 https://etherscan.io/address/0x1c8f6a5f009e051cab9c3851ca2da2c936b2775a
----
2023-03-08 16:21:23 sell $1,556.25 1 57.015 88,525.9 57.015 https://etherscan.io/address/0xa32394b885da5379b8459ca9a01146de8a70d49d
----
2023-03-08 16:13:11 buy $1,558.27 1 1

In [26]:
#now the way with while loop for more pages
# specify the URL of the WETH/USDT page
url = 'https://www.dextools.io/app/en/ether/pair-explorer/0x4e68ccd3e89f51c3074ca5072bbac773960dfa36'

options = Options()
#options.add_experimental_option("detach", True)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                          options=options)

driver.get(url)

page = 0

all_results = []  # list for all rows

while True:

    page += 1
    print('--- page:', page, '---')
    
    # get table
    tableElement = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, 'ngx-datatable'))
    )
    # scroll into table view
    driver.execute_script("arguments[0].scrollIntoView();", tableElement)

    # scrolling through the table body to the bottom
    tableBodyelement = tableElement.find_element(By.TAG_NAME, "datatable-body-cell")
    driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight)", tableBodyelement)

    rowWrapper = tableElement.find_elements(By.TAG_NAME, 'datatable-row-wrapper')

    for row in rowWrapper:
        cells = row.find_elements(By.TAG_NAME, 'datatable-body-cell')
        date = cells[0].text
        type = cells[1].text
        price_usd = cells[2].text
        price_eth = cells[3].text
        amount_coin_1 = cells[4].text
        total_coin_2 = cells[5].text  #the naming here should be done better, it will depend on the pairs I will be trading
        total_chain_coin = cells[6].text
        maker = cells[7].find_element(By.TAG_NAME, 'a').get_attribute('href')
        #print(date, type, price_usd, price_eth, amount_coin_1, total_coin_2, total_chain_coin, maker)
        #print('----')  
        
        # add row to list
        all_results.append( [date, type, price_usd, price_eth, amount_coin_1, total_coin_2, total_chain_coin, maker] )
                                

    try:
        next_page = driver.find_element(By.XPATH, '//a[@aria-label="go to next page"]')
        #next_page.click()
        driver.execute_script("arguments[0].click();", next_page) #this should solve the issue of not finding the element?
        time.sleep(0.5)
    except Exception as ex:
       print("last page???")
       break
    
# after loop convert to DataFrame and write it to excel


#print(all_results)
#import pandas as pd

#df = pd.DataFrame(all_results, columns=['date', 'type', 'price_usd', 'price_eth', 'amount_cuminu', 'total_eth', 'maker'])
#df.to_excel('results.xlsx')

# ---


#note, TODO next: it scrapes well, just at the end there is a problem that it cycles and runs the same set of table rows in a loop. 
# Resolve next time, look how the button changes at the end, if not available - perhaps if block?


--- page: 1 ---
--- page: 2 ---
--- page: 3 ---
--- page: 4 ---
--- page: 5 ---
--- page: 6 ---


Exception ignored in: <function Service.__del__ at 0x000001F891E1FD00>
Traceback (most recent call last):
  File "d:\Dokumenty\Vejška\Magisterské studium\DIPLOMKA\Code_and_Data\Data_scraping\DEX_data_scraper\venv\lib\site-packages\selenium\webdriver\common\service.py", line 183, in __del__
    self.stop()
  File "d:\Dokumenty\Vejška\Magisterské studium\DIPLOMKA\Code_and_Data\Data_scraping\DEX_data_scraper\venv\lib\site-packages\selenium\webdriver\common\service.py", line 149, in stop
    self.send_remote_shutdown_command()
  File "d:\Dokumenty\Vejška\Magisterské studium\DIPLOMKA\Code_and_Data\Data_scraping\DEX_data_scraper\venv\lib\site-packages\selenium\webdriver\common\service.py", line 133, in send_remote_shutdown_command
    if not self.is_connectable():
  File "d:\Dokumenty\Vejška\Magisterské studium\DIPLOMKA\Code_and_Data\Data_scraping\DEX_data_scraper\venv\lib\site-packages\selenium\webdriver\common\service.py", line 122, in is_connectable
    return utils.is_connectable(self.po

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=110.0.5481.180)
Stacktrace:
Backtrace:
	(No symbol) [0x00DB37D3]
	(No symbol) [0x00D48B81]
	(No symbol) [0x00C4B36D]
	(No symbol) [0x00C2E5DA]
	(No symbol) [0x00C9F0CB]
	(No symbol) [0x00CB0A56]
	(No symbol) [0x00C9B216]
	(No symbol) [0x00C70D97]
	(No symbol) [0x00C7253D]
	GetHandleVerifier [0x0102ABF2+2510930]
	GetHandleVerifier [0x01058EC1+2700065]
	GetHandleVerifier [0x0105C86C+2714828]
	GetHandleVerifier [0x00E63480+645344]
	(No symbol) [0x00D50FD2]
	(No symbol) [0x00D56C68]
	(No symbol) [0x00D56D4B]
	(No symbol) [0x00D60D6B]
	BaseThreadInitThunk [0x76346BD9+25]
	RtlGetFullPathName_UEx [0x773C8FD2+1218]
	RtlGetFullPathName_UEx [0x773C8F9D+1165]


In [4]:
print(len(all_results))

15


In [24]:
#Just try to find an element and click it
# specify the URL of the WETH/USDT page
url = 'https://www.dextools.io/app/en/ether/pair-explorer/0x4e68ccd3e89f51c3074ca5072bbac773960dfa36'

options = Options()
#options.add_experimental_option("detach", True)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                          options=options)

driver.get(url)
time.sleep(5)
next_page = driver.find_element(By.XPATH, '//a[@aria-label="go to next page"]')



print(next_page.get_attribute("innerHTML"))
#next_page.click()
driver.execute_script("arguments[0].click();", next_page)
#print(test_elemnt.get_attribute("innerHTML"))



<i class="datatable-icon-right"></i>
